# Kubernetes Llama2 ChatBot 



The Python package provides simple bindings for the llama.cpp library, offering access to the C API via ctypes interface, a high-level Python API for text completion, OpenAI-like API, and LangChain compatibility. It supports multiple BLAS backends for faster processing and includes both high-level and low-level APIs, along with web server functionality.

`llama.cpp`'s objective is to run the LLaMA model with 4-bit integer quantization on MacBook. It is a plain C/C++ implementation optimized for Apple silicon and x86 architectures, supporting various integer quantization and BLAS libraries. Originally a web chat example, it now serves as a development playground for ggml library features.

`GGML`, a C library for machine learning, facilitates the distribution of large language models (LLMs). It utilizes quantization to enable efficient LLM execution on consumer hardware. GGML files contain binary-encoded data, including version number, hyperparameters, vocabulary, and weights. The vocabulary comprises tokens for language generation, while the weights determine the LLM's size. Quantization reduces precision to optimize resource usage.

In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.3 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.8/807.8 kB 7.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 9.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 59.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 18.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 6.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 8.4 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-msjyuoxh/overlay/local/bin
    changing mode of /tmp/pip-build-env-msjyuoxh/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-msjyuoxh/overlay/local

The GGML format has been replaced by GGUF, effective as of August 21st, 2023. Starting from this date, llama.cpp will no longer provide compatibility with GGML models. This notebook uses `llama-cpp-python==0.1.78, which is compatible with GGML Models`.

In [2]:
# For download the models
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.4 MB/s eta 0:00:00


## Downloading the Model

In this case, we will use the model called [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML).

In [3]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

First, we download the model

In [4]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

### Inference with llama-cpp-python

Loading the model

In [5]:
# GPU
from llama_cpp import Llama
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096, # Context window
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [6]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

43

We will use this prompt.

In [7]:
prompt = "Write a linear regression in python"
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

Generating response

In [8]:
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=256,
    temperature=0.5,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop = ['USER:'], # Dynamic stopping when such token is detected.
    echo=True # return the prompt
)

print(response["choices"][0]["text"])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression in python

ASSISTANT:

To write a linear regression in Python, you can use the scikit-learn library. Here is an example of how to do this:
```
from sklearn.linear_model import LinearRegression
import pandas as pd

# Load your data into a Pandas dataframe
df = pd.read_csv('your_data.csv')

# Create a linear regression object and fit the data
reg = LinearRegression().fit(df[['x1', 'x2']], df['y'])

# Print the coefficients
print(reg.coef_)

# Print the R-squared value
print(reg.score(df[['x1', 'x2']], df['y']))
```
This code will load your data into a Pandas dataframe, create a linear regression object and fit the data using the `fit()` method. It will then print the coefficients of the linear regression and the R-squared value, which measures the goodness of fit of the model.

Please note that you need to replace `'your_data.csv'` with the name of your own dataset fil

### Inference with langchain




In [15]:
!pip -q install langchain gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 8.6 MB/s eta 0:00:00


Due to vRAM limitations, before running these cells, you need to delete the previous model from memory.

In [10]:
lcpp_llm.reset()
lcpp_llm.set_cache(None)
lcpp_llm = None
del lcpp_llm


Prompt for the model.

In [11]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from huggingface_hub import hf_hub_download

template = """''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.
USER: {question}
ASSISTANT: """
prompt = PromptTemplate(template=template, input_variables=["question"])

Stream tokens

In [12]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

Load the model

In [13]:
n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Loading model,
llm = LlamaCpp(
    model_path=model_path,
    max_tokens=1024,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,
    n_ctx=4096, # Context window
    stop = ['USER:'], # Dynamic stopping when such token is detected.
    temperature = 0.4,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


Generating response

In [14]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Write a simple linear regression in python"

llm_chain.run(question)

 Sure thing! Here's an example of how you could write a simple linear regression in Python using scikit-learn library:
```
from sklearn.linear_model import LinearRegression
import pandas as pd

# Load your dataset
df = pd.read_csv('your_data.csv')

# Create X and y variables
X = df[['feature1', 'feature2']]
y = df['target']

# Create a linear regression object and fit the data
reg = LinearRegression()
reg.fit(X, y)

# Print the coefficients
print(reg.coef_)

# Print the R-squared value
print(reg.score(X, y))
```
This code assumes that your dataset is stored in a CSV file called `your_data.csv`, and that you have two features (feature1 and feature2) and one target variable (target). It creates X and y variables using the columns of the dataframe, fits a linear regression model to the data using the `fit()` method, prints the coefficients of the model using the `coef_` attribute, and prints the R-squared value using the `score()` method.

I hope this helps! Let me know if you have any qu

" Sure thing! Here's an example of how you could write a simple linear regression in Python using scikit-learn library:\n```\nfrom sklearn.linear_model import LinearRegression\nimport pandas as pd\n\n# Load your dataset\ndf = pd.read_csv('your_data.csv')\n\n# Create X and y variables\nX = df[['feature1', 'feature2']]\ny = df['target']\n\n# Create a linear regression object and fit the data\nreg = LinearRegression()\nreg.fit(X, y)\n\n# Print the coefficients\nprint(reg.coef_)\n\n# Print the R-squared value\nprint(reg.score(X, y))\n```\nThis code assumes that your dataset is stored in a CSV file called `your_data.csv`, and that you have two features (feature1 and feature2) and one target variable (target). It creates X and y variables using the columns of the dataframe, fits a linear regression model to the data using the `fit()` method, prints the coefficients of the model using the `coef_` attribute, and prints the R-squared value using the `score()` method.\n\nI hope this helps! Let m

In [16]:
import gradio as gr
# Create generate function - this will be called when a user runs the gradio app
def generate(prompt):
    # The prompt will get passed to the LLM Chain!
    return llm_chain.run(prompt)
    # And will return responses

# Define a string variable to hold the title of the app
title = '🦜🔗 Chatbot LLama2 GGML'
# Define another string variable to hold the description of the app
description = 'Chatbot LLama2 GGML'


In [ ]:
# Build gradio interface, define inputs and outputs...just text in this
gr.Interface(fn=generate, inputs=["text"], outputs=["text"],
             # Pass through title and description
             title=title, description=description).launch(server_port=8083
                                                          , share=True)

- GPTQ is a specific format for GPU only.

- GGML is designed for CPU and Apple M series but can also offload some layers on the GPU

- GGMLs like q4_2, q4_3, q5_0, q5_1 and q8_0 have superior inference quality and outperform GPTQ on benchmarks